# T81-558: Applications of Deep Neural Networks
**Module 12: Deep Learning and Security**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module Video Material

Main video lecture:

* Part 12.1: Introduction to the OpenAI Gym [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* Part 12.2: Introduction to Q-Learning for Keras [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* **Part 12.3: Keras Q-Learning in the OpenAI Gym** [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* Part 12.4: Atari Games with Keras Neural Networks [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* 12.5: How Alpha Zero used Reinforcement Learning to Master Chess [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)


# Part 12.3: Keras Q-Learning in the OpenAI Gym

![Deep Q-Learning](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/deepqlearning.png "Reinforcement Learning")

* **CEMAgent**
    * **model** - The neural network that will be trained.
    * **nb_actions** - The number of actions the agent can take (e.g. up, down, left, right, fire)
    * **memory** - The EpisodeParameterMemory object to use.  This object observes and save all of the state transitions so that you can train your network on them later on (instead of having to make observations from the environment all the time).
    * **batch_size** - The batch size for neural network training, same concept as deep learning batch sizes.
    * **nb_steps_warmup** - Number of training steps to pass before any learning occurs.
    * **train_interval** - Logging interval, defines how often to log.
    * **elite_frac**
* **CEMAgent.fit**
    * **env** - The OpenAI gym environment being used.
    * **nb_steps** - Number of training steps to be performed.
    * **visualize** - If `True`, the environment is visualized during training. However,
                this is likely going to slow down training significantly and is thus intended to be
                a debugging instrument.
    * **verbose** - 0 for no logging, 1 for interval logging (compare `log_interval`), 2 for episode logging




In [1]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

ENV_NAME = 'CartPole-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

# Option 2: deep network
# model = Sequential()
# model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(nb_actions))
# model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in tensorflow.keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=1000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem.compile()

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the best weights.
cem.save_weights('cem_{}_params.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None
Training for 100000 steps ...
    57/100000: episode: 1, duration: 0.067s, episode steps:  57, steps per second: 853, episode reward: 57.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.491 [0.000, 1.000],  mean_best_reward: --
    81/100000: episode: 2, duration: 0.012s, episode steps:  24, steps per second: 1994, episode reward: 24.000, mean reward:  1.000 [ 1.000,  1.000

   909/100000: episode: 38, duration: 0.010s, episode steps:  17, steps per second: 1664, episode reward: 17.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.412 [0.000, 1.000],  mean_best_reward: --
   932/100000: episode: 39, duration: 0.014s, episode steps:  23, steps per second: 1660, episode reward: 23.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.478 [0.000, 1.000],  mean_best_reward: --
   946/100000: episode: 40, duration: 0.013s, episode steps:  14, steps per second: 1093, episode reward: 14.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.714 [0.000, 1.000],  mean_best_reward: --
   965/100000: episode: 41, duration: 0.015s, episode steps:  19, steps per second: 1284, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.579 [0.000, 1.000],  mean_best_reward: --
   984/100000: episode: 42, duration: 0.015s, episode steps:  19, steps per second: 1287, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action:

  1878/100000: episode: 86, duration: 0.014s, episode steps:  22, steps per second: 1597, episode reward: 22.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  1895/100000: episode: 87, duration: 0.010s, episode steps:  17, steps per second: 1648, episode reward: 17.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.294 [0.000, 1.000],  mean_best_reward: --
  1910/100000: episode: 88, duration: 0.010s, episode steps:  15, steps per second: 1574, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.400 [0.000, 1.000],  mean_best_reward: --
  1932/100000: episode: 89, duration: 0.012s, episode steps:  22, steps per second: 1880, episode reward: 22.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.409 [0.000, 1.000],  mean_best_reward: --
  1944/100000: episode: 90, duration: 0.010s, episode steps:  12, steps per second: 1152, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], mean action:

  3080/100000: episode: 133, duration: 0.012s, episode steps:  23, steps per second: 1893, episode reward: 23.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.565 [0.000, 1.000],  mean_best_reward: --
  3136/100000: episode: 134, duration: 0.030s, episode steps:  56, steps per second: 1851, episode reward: 56.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  3157/100000: episode: 135, duration: 0.016s, episode steps:  21, steps per second: 1338, episode reward: 21.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.524 [0.000, 1.000],  mean_best_reward: --
  3198/100000: episode: 136, duration: 0.021s, episode steps:  41, steps per second: 1962, episode reward: 41.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.512 [0.000, 1.000],  mean_best_reward: --
  3216/100000: episode: 137, duration: 0.010s, episode steps:  18, steps per second: 1885, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean ac

  4519/100000: episode: 175, duration: 0.023s, episode steps:  20, steps per second: 887, episode reward: 20.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.550 [0.000, 1.000],  mean_best_reward: --
  4549/100000: episode: 176, duration: 0.033s, episode steps:  30, steps per second: 919, episode reward: 30.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  4577/100000: episode: 177, duration: 0.027s, episode steps:  28, steps per second: 1032, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  4636/100000: episode: 178, duration: 0.057s, episode steps:  59, steps per second: 1037, episode reward: 59.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.525 [0.000, 1.000],  mean_best_reward: --
  4705/100000: episode: 179, duration: 0.067s, episode steps:  69, steps per second: 1031, episode reward: 69.000, mean reward:  1.000 [ 1.000,  1.000], mean acti

...

 92361/100000: episode: 2533, duration: 0.012s, episode steps:  20, steps per second: 1719, episode reward: 20.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.600 [0.000, 1.000],  mean_best_reward: --
 92435/100000: episode: 2534, duration: 0.035s, episode steps:  74, steps per second: 2085, episode reward: 74.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 92454/100000: episode: 2535, duration: 0.010s, episode steps:  19, steps per second: 1936, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.316 [0.000, 1.000],  mean_best_reward: --
 92478/100000: episode: 2536, duration: 0.012s, episode steps:  24, steps per second: 2054, episode reward: 24.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.458 [0.000, 1.000],  mean_best_reward: --
 92516/100000: episode: 2537, duration: 0.018s, episode steps:  38, steps per second: 2131, episode reward: 38.000, mean reward:  1.000 [ 1.000,  1.000], me

 94092/100000: episode: 2579, duration: 0.028s, episode steps:  60, steps per second: 2149, episode reward: 60.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 94117/100000: episode: 2580, duration: 0.014s, episode steps:  25, steps per second: 1795, episode reward: 25.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.480 [0.000, 1.000],  mean_best_reward: --
 94158/100000: episode: 2581, duration: 0.019s, episode steps:  41, steps per second: 2134, episode reward: 41.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.488 [0.000, 1.000],  mean_best_reward: --
 94177/100000: episode: 2582, duration: 0.010s, episode steps:  19, steps per second: 1925, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.474 [0.000, 1.000],  mean_best_reward: --
 94187/100000: episode: 2583, duration: 0.005s, episode steps:  10, steps per second: 1869, episode reward: 10.000, mean reward:  1.000 [ 1.000,  1.000], me

 95785/100000: episode: 2626, duration: 0.048s, episode steps: 100, steps per second: 2071, episode reward: 100.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.480 [0.000, 1.000],  mean_best_reward: --
 95805/100000: episode: 2627, duration: 0.011s, episode steps:  20, steps per second: 1835, episode reward: 20.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 95832/100000: episode: 2628, duration: 0.014s, episode steps:  27, steps per second: 1950, episode reward: 27.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.481 [0.000, 1.000],  mean_best_reward: --
 95857/100000: episode: 2629, duration: 0.012s, episode steps:  25, steps per second: 2020, episode reward: 25.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.520 [0.000, 1.000],  mean_best_reward: --
 95869/100000: episode: 2630, duration: 0.007s, episode steps:  12, steps per second: 1841, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], m

 97429/100000: episode: 2677, duration: 0.015s, episode steps:  28, steps per second: 1836, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.464 [0.000, 1.000],  mean_best_reward: --
 97454/100000: episode: 2678, duration: 0.015s, episode steps:  25, steps per second: 1614, episode reward: 25.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.440 [0.000, 1.000],  mean_best_reward: --
 97476/100000: episode: 2679, duration: 0.012s, episode steps:  22, steps per second: 1810, episode reward: 22.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.545 [0.000, 1.000],  mean_best_reward: --
 97523/100000: episode: 2680, duration: 0.023s, episode steps:  47, steps per second: 2031, episode reward: 47.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.468 [0.000, 1.000],  mean_best_reward: --
 97605/100000: episode: 2681, duration: 0.039s, episode steps:  82, steps per second: 2129, episode reward: 82.000, mean reward:  1.000 [ 1.000,  1.000], me

 99147/100000: episode: 2718, duration: 0.041s, episode steps:  85, steps per second: 2082, episode reward: 85.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.482 [0.000, 1.000],  mean_best_reward: --
 99177/100000: episode: 2719, duration: 0.015s, episode steps:  30, steps per second: 1957, episode reward: 30.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.533 [0.000, 1.000],  mean_best_reward: --
 99226/100000: episode: 2720, duration: 0.024s, episode steps:  49, steps per second: 2033, episode reward: 49.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.510 [0.000, 1.000],  mean_best_reward: --
 99273/100000: episode: 2721, duration: 0.023s, episode steps:  47, steps per second: 2047, episode reward: 47.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.532 [0.000, 1.000],  mean_best_reward: --
 99298/100000: episode: 2722, duration: 0.013s, episode steps:  25, steps per second: 1906, episode reward: 25.000, mean reward:  1.000 [ 1.000,  1.000], me